<a href="https://colab.research.google.com/github/trungduyen0220/text-classification-assignment/blob/master/Assignment1_NguyenThiMyDuyen_MSE13028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%tensorflow_version 2.x

# Dataset

In [1]:
!rm -f train.csv test.csv
!wget https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/train.csv
!wget https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/test.csv

--2020-09-06 15:33:02--  https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10400749 (9.9M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   9.92M  19.9MB/s    in 0.5s    

2020-09-06 15:33:03 (19.9 MB/s) - ‘train.csv’ saved [10400749/10400749]

--2020-09-06 15:33:03--  https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1133794 (1.1M) [text/plain]
Saving

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
import csv
import zipfile
import glob
from string import digits

import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus.reader.wordnet import NOUN, VERB, ADJ
from sklearn.feature_extraction.text import CountVectorizer
english_stops = set(stopwords.words('english'))

# from keras.models import Model
# from keras.layers import Dense, Input, Dropout, LSTM, Activation
# from keras.layers.embeddings import Embedding
# from keras.preprocessing import sequence
# from keras.initializers import glorot_uniform

from collections import defaultdict
import math

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,title,abstract,category
0,1901.05003,Quantum Teleportation-Inspired Algorithm for S...,We show that low-depth random quantum circui...,quant-ph
1,1903.08905,RAP-Net: Recurrent Attention Pooling Networks ...,The response selection has been an emerging ...,cs
2,1907.11533,Combinatorial protein-protein interactions on ...,Scaffold proteins organize cellular processe...,q-bio
3,1711.07582,CVXR: An R Package for Disciplined Convex Opti...,CVXR is an R package that provides an object...,stat
4,1910.05833,Analytical Solution of (2+1) Dimensional Dirac...,"In this article, we studied the system of (2...",quant-ph


In [13]:
def read_csv(filename):
    abstract = []
    category = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile )

        for row in csvReader:
            abstract.append(row[2])
            category.append(row[3])

    abstract.pop(0)
    category.pop(0)
    # category = pd.factorize(category)[0]
    X = np.asarray(abstract)
    Y = np.asarray(category)

    return X, Y

X_train, Y_train = read_csv('train.csv')
X_test, Y_test = read_csv('test.csv')

In [14]:
def preformat(sentence):
  sentence = sentence.lower()
  sentence = sentence.translate(str.maketrans('', '', digits))
  sentence = re.sub(r"http\S+", "", sentence)
  sentence = sentence.translate(str.maketrans('', '', string.punctuation))
  sentence = sentence.translate ({ord(c): " " for c in "!@#$%^&*()[]{}/<>?\\|`~=_+'"""})

  tokens = word_tokenize(sentence)
  tags = nltk.pos_tag(tokens)

  lemmatizer = WordNetLemmatizer()
  words = ""
  for i, token in enumerate(tokens):
    pos_tag = tags[i][1]
    if pos_tag.startswith('N'):
        lemma = lemmatizer.lemmatize(token, pos=NOUN)
        words += lemma + " "
    elif pos_tag.startswith('V'):
        lemma = lemmatizer.lemmatize(token, pos=VERB)
        words += lemma + " "
    elif pos_tag.startswith('J'):
        lemma = lemmatizer.lemmatize(token, pos=ADJ)
        words += lemma + " "
    else:
        lemma = token
        words += " "
  
  return ' '.join(words.split())

In [15]:
def build_vocab(X):
    """Build vocabulary from dataset

    Args:
        texts (list): list of tokenized sentences
    
    Returns:
        vocab (dict): map from word to index
    """
    vocab = {}
    for sentences in X:
        sentences = preformat(sentences)
        for word in sentences.split():
            # Check if word is a punctuation
            
            if word in string.punctuation:
                continue
            if word not in vocab:
                idx = len(vocab)
                vocab[word] = idx
    return vocab

In [16]:

def train_naive_bayes(texts, labels, target_classes, alpha=1):
    """Train a multinomial Naive Bayes model
    """
    ndoc = 0
    nc = defaultdict(int)   # map from a class label to number of documents in the class
    logprior = dict()
    loglikelihood = dict()
    count = defaultdict(int)  # count the occurrences of w in documents of class c

    vocab = build_vocab(texts)
    # Training
    for s, c in zip(texts, labels):
        ndoc += 1
        nc[c] += 1
        for w in s.split():
            if w in vocab:
                count[(w,c)] += 1
    
    vocab_size = len(vocab)
    for c in target_classes:
        logprior[c] = math.log(nc[c]/ndoc)
        sum_ = 0
        for w in vocab.keys():
            if (w,c) not in count: count[(w,c)] = 0
            sum_ += count[(w,c)]
        
        for w in vocab.keys():
            loglikelihood[(w,c)] = math.log( (count[(w,c)] + alpha) / (sum_ + alpha * vocab_size) )
    
    return logprior, loglikelihood, vocab

In [17]:
target_classes = set()
for index in Y_train:
    target_classes.add(index)

print(target_classes)

{'cond-mat', 'eess', 'cs', 'hep-ph', 'astro-ph', 'hep-th', 'math', 'quant-ph', 'q-bio', 'stat'}


In [18]:
logprior, loglikelihood, vocab = train_naive_bayes(X_train, Y_train, target_classes)

In [19]:
def test_naive_bayes(testdoc, logprior, loglikelihood, target_classes, vocab):
    sum_ = {}
    for c in  target_classes:
        sum_[c] = logprior[c]
        for w in testdoc.split():
            if w in vocab:
                sum_[c] += loglikelihood[(w,c)]
    # sort keys in sum_ by value
    sorted_keys = sorted(sum_.keys(), key=lambda x: sum_[x], reverse=True)
    return sorted_keys[0]

In [20]:
print(target_classes)
print(vocab)
print()
for index in range(10):
    print(str(Y_train[index]))

{'cond-mat', 'eess', 'cs', 'hep-ph', 'astro-ph', 'hep-th', 'math', 'quant-ph', 'q-bio', 'stat'}
{'show': 0, 'lowdepth': 1, 'random': 2, 'quantum': 3, 'circuit': 4, 'be': 5, 'simulate': 6, 'teleportationinspired': 7, 'algorithm': 8, 'use': 9, 'logical': 10, 'qubits': 11, 'redirect': 12, 'teleport': 13, 'information': 14, 'original': 15, 'renormalize': 16, 'new': 17, 'small': 18, 'number': 19, 'demonstrate': 20, 'several': 21, 'include': 22, 'dchain': 23, 'qubit': 24, 'depth': 25, 'dgrid': 26, 'dbristlecone': 27, 'result': 28, 'present': 29, 'memoryefficient': 30, 'method': 31, 'clear': 32, 'physical': 33, 'picture': 34, 'response': 35, 'selection': 36, 'have': 37, 'emerge': 38, 'research': 39, 'topic': 40, 'due': 41, 'grow': 42, 'interest': 43, 'dialogue': 44, 'modeling': 45, 'goal': 46, 'task': 47, 'select': 48, 'appropriate': 49, 'continue': 50, 'further': 51, 'push': 52, 'endtoend': 53, 'model': 54, 'realworld': 55, 'scenario': 56, 'seventh': 57, 'dialog': 58, 'system': 59, 'technolo

In [41]:
for index in range(10):
  print(str(Y_train[index]) + " " + test_naive_bayes(X_train[index], logprior, loglikelihood, target_classes, vocab))

quant-ph quant-ph
cs cs
q-bio q-bio
stat eess
quant-ph hep-th
q-bio q-bio
hep-ph hep-ph
quant-ph quant-ph
quant-ph quant-ph
hep-ph hep-ph


In [22]:
predicted_labels = [test_naive_bayes(s, logprior, loglikelihood, target_classes, vocab)
                    for s in X_test]
from sklearn import metrics

print('Accuracy score: %f' % metrics.accuracy_score(Y_test, predicted_labels))

Accuracy score: 0.865000


In [26]:
print('Evaluation measures for class %s' % c)
print('  Precision: %f' % metrics.precision_score(Y_test, predicted_labels, average='micro'))
print('  Recall: %f' % metrics.recall_score(Y_test, predicted_labels, average='micro'))
print('  F1: %f' % metrics.f1_score(Y_test, predicted_labels, average='micro'))

Evaluation measures for class stat
  Precision: 0.865000
  Recall: 0.865000
  F1: 0.865000


In [27]:
print('Macro-averaged f1: %f' % metrics.f1_score(Y_test, predicted_labels, average='macro'))
print('Micro-averaged f1: %f' % metrics.f1_score(Y_test, predicted_labels, average='micro'))

Macro-averaged f1: 0.865216
Micro-averaged f1: 0.865000


In [44]:
print(metrics.classification_report(Y_test, predicted_labels, digits=3))

              precision    recall  f1-score   support

    astro-ph      0.941     0.950     0.945       100
    cond-mat      0.864     0.890     0.877       100
          cs      0.694     0.770     0.730       100
        eess      0.811     0.730     0.768       100
      hep-ph      0.956     0.870     0.911       100
      hep-th      0.865     0.900     0.882       100
        math      0.920     0.810     0.862       100
       q-bio      0.885     0.920     0.902       100
    quant-ph      0.891     0.900     0.896       100
        stat      0.850     0.910     0.879       100

    accuracy                          0.865      1000
   macro avg      0.868     0.865     0.865      1000
weighted avg      0.868     0.865     0.865      1000

